# Morphologic Transformations Visualizations 🔲

This notebook contains algorithms for visualizing some basic morphologic transformations such as erosion, dilation, opening, and closing. 

It serves the purpose of offering a better understanding in the process of exploring Image Processing. 📚

In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Dilation

In [2]:
input_img = cv.imread("./img/mon1thr1_bw.bmp", cv.IMREAD_GRAYSCALE)
cv.imshow("input", input_img)
cv.waitKey(0)                    # press on any key to close the image
cv.destroyWindow("input") 

## Out-of-box behavior

In [3]:
cv.imshow("source", input_img)

structuring_elem = cv.getStructuringElement(cv.MORPH_CROSS, ksize=(3, 3))
default_dilated = cv.dilate(input_img, structuring_elem, iterations = 1)
default_eroded = cv.erode(input_img, structuring_elem, iterations = 1)
cv.imshow("default dilation", default_dilated)
cv.imshow("default eroded", default_eroded)

cv.waitKey(0)
cv.destroyWindow("source")
cv.destroyWindow("default dilation")
cv.destroyWindow("default eroded")